In [1]:
# Standard imports 
import pandas as pd
import glob
import numpy as np 
import math 
from datetime import datetime
from bokeh.plotting import figure, output_notebook, show

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
# Read in all the csv files from 
path =r'/Users/yogadude/Desktop/Main Folder/bc9_datascience/WebScrapingProject' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
frame = pd.concat(list_)

In [4]:
# Some standard EDA 
#look at the DF
frame = frame.reindex()
frame.head()

,category,author,title,num_comments,tags,article_url,time,recs_urls
0,life,Graham Gremore,Bigotry is winning? LGBTQ people are retreatin...,NaN,life|antigay|bigotry|closeted|discrimination|h...,https://www.queerty.com/bigotry-winning-lgbtq-...,2017-05-02T14:05:51+00:00,http://saveondentalimplant.com/?utm_source=tbo...
1,life,NaN,The National Equality March is right around th...,3.0,life|facebook|nicole-murray ramirez|pride|san ...,https://www.queerty.com/national-equality-marc...,2017-05-02T13:38:32+00:00,http://saveondentalimplant.com/?utm_source=tbo...
2,life,NaN,"Will Young has sent out so many private pics, ...",5.0,life|the week in leaks|will young,https://www.queerty.com/will-young-sent-many-p...,2017-05-02T13:05:34+00:00,http://saveondentalimplant.com/?utm_source=tbo...
3,life,Derek de Koff,Study: Straight men are more attracted to thei...,13.0,life|alcohol|gay|gay sex|straight|study,https://www.queerty.com/study-straight-men-att...,2017-05-02T11:00:52+00:00,http://definition.org/waltons-family-happened/...
4,life,NaN,Heres how Matt Bomer came out to his family,3.0,life|coming out|matt bomer,https://www.queerty.com/heres-matt-bomer-came-...,2017-05-01T16:22:50+00:00,http://www.historyinorbit.com/russias-new-gene...


In [5]:
# the column names 
frame.columns

Index([u'category', u'author', u'title', u'num_comments', u'tags',
       u'article_url', u'time', u'recs_urls'],
      dtype='object')

In [6]:
frame.dtypes #convert time to datetime object

category         object
author           object
title            object
num_comments    float64
tags             object
article_url      object
time             object
recs_urls        object
dtype: object

In [7]:
# Dimensions of the Dataframe 
print frame.shape 
frame.dropna(axis=0, how ="any", inplace = True)
print frame.shape

(8109, 8)
(5110, 8)


In [8]:
frame.describe() # not much numerical data tho

,num_comments
count,5110.000000
mean,14.267319
std,21.587717
min,1.000000
25%,4.000000
50%,8.000000
75%,17.000000
max,357.000000


In [9]:
frame["category"].value_counts()

entertainment    3327
politics         1415
life              368
Name: category, dtype: int64

In [10]:
#Convert the Time to a datetime object
frame['time'] = frame['time'].apply(lambda x: x.split('T')[0])

In [11]:
frame['time'] = pd.to_datetime(frame['time'])

In [12]:
frame.dtypes

category                object
author                  object
title                   object
num_comments           float64
tags                    object
article_url             object
time            datetime64[ns]
recs_urls               object
dtype: object

In [13]:
# Make a copy of the original dataframe
df_urls = frame.copy()
# Split the column the column using '|'
df_urls.recs_urls = df_urls.recs_urls.apply(lambda x: x.split('|'))

In [14]:
df_urls.recs_urls.head() # How many urls are in the reccommended urls and make the length of the list a new feature

3    [http://definition.org/waltons-family-happened...
5    [http://definition.org/waltons-family-happened...
6    [http://definition.org/waltons-family-happened...
7    [http://www.lifedaily.com/story/someone-spotte...
8    [http://www.historyinorbit.com/russias-new-gen...
Name: recs_urls, dtype: object

In [15]:
print df_urls.recs_urls.apply(lambda x: len(x)).head()
print df_urls.recs_urls.apply(lambda x: len(x)).tail()

3    36
5    36
6    36
7    36
8    36
Name: recs_urls, dtype: int64
2128    16
2129    16
2131    16
2133    16
2134    16
Name: recs_urls, dtype: int64


In [16]:
df_urls = pd.concat([pd.DataFrame(df_urls.iloc[i].to_dict()) for i in range(len(df_urls))])

In [17]:
df_urls.shape # it works 

(143080, 8)

In [18]:
df_urls.head()
# Create new columns of the attributes 
# Double the columns text_x and text_y 
# Split the tags columns of the merged dataframe to get a list of tags
# convert to list to a set and find the intesection of these two sets. which gives the commonality 

,article_url,author,category,num_comments,recs_urls,tags,time,title
0,https://www.queerty.com/study-straight-men-att...,Derek de Koff,life,13.0,http://definition.org/waltons-family-happened/...,life|alcohol|gay|gay sex|straight|study,2017-05-02,Study: Straight men are more attracted to thei...
1,https://www.queerty.com/study-straight-men-att...,Derek de Koff,life,13.0,http://definition.org/waltons-family-happened/...,life|alcohol|gay|gay sex|straight|study,2017-05-02,Study: Straight men are more attracted to thei...
2,https://www.queerty.com/study-straight-men-att...,Derek de Koff,life,13.0,https://www.queerty.com/straight-guys-confess-...,life|alcohol|gay|gay sex|straight|study,2017-05-02,Study: Straight men are more attracted to thei...
3,https://www.queerty.com/study-straight-men-att...,Derek de Koff,life,13.0,https://www.queerty.com/straight-guys-confess-...,life|alcohol|gay|gay sex|straight|study,2017-05-02,Study: Straight men are more attracted to thei...
4,https://www.queerty.com/study-straight-men-att...,Derek de Koff,life,13.0,http://www.lifedaily.com/story/someone-spotted...,life|alcohol|gay|gay sex|straight|study,2017-05-02,Study: Straight men are more attracted to thei...


In [20]:
# Merge the dataframe with itself based on the article url left inner 
df_urls2 = df_urls.copy()
df_merged = pd.merge(df_urls, df_urls2, on="article_url") # default is inner join but is this correct
#Shouldn't it be a merging on the recommended urls? We see from a few lines down in the code that the 
# intersection of all the sets are the same if merged on the original article url. Let's try it. 
#df_merged = pd.merge(df_urls, df_urls2, on="recs_urls")
df_merged.head()
df_merged.shape

(4504000, 15)

In [21]:
# Split the tags columns of the merged dataframe to get a list of tags
df_merged.tags_x = df_merged.tags_x.apply(lambda x: x.split('|'))

In [22]:
df_merged.tags_y = df_merged.tags_y.apply(lambda x: x.split('|'))

In [23]:
#checking the tags are now a list and convert to sets
df_merged['tags_x'] = df_merged['tags_x'].apply(set)

In [24]:
df_merged['tags_y'] = df_merged['tags_y'].apply(set)

In [27]:
df_merged[['tags_x','tags_y']].tail()

,tags_x,tags_y
4503995,"{politics, art, uk}","{politics, art, uk}"
4503996,"{politics, art, uk}","{politics, art, uk}"
4503997,"{politics, art, uk}","{politics, art, uk}"
4503998,"{politics, art, uk}","{politics, art, uk}"
4503999,"{politics, art, uk}","{politics, art, uk}"


In [ ]:
df_merged['intersect'] = df_merged.apply(lambda x: (x['tags_x'] & x['tags_y']), axis=1) # this takes a while

In [ ]:
# Is there any differences seen in the tags? 
len(df_merged['tags_x'] == df_merged['tags_y']) # No, the tags are the same

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Does an author get more comments from readers than others? 
frame.groupby('author')['num_comments'].mean().sort_values(ascending=True).head(20).plot.barh()
plt.xlabel('Averge Number of Comments')

In [ ]:
# Does an author get more comments from readers than others? 
frame.groupby(['category'])['num_comments'].mean().sort_values(ascending=True).plot.bar()
plt.ylabel('Averge Number of Comments')

In [ ]:
# Is there any correlation with the title length and the number of comments? 
frame['title_length'] = frame['title'].str.split().str.len()
sns.plt.scatter(frame['title_length'], frame['num_comments'],c='r')
# frame.groupby(['category'])['num_comments'].mean().sort_values(ascending=True).plot.bar()
plt.xlabel('Title Length')
plt.ylabel('Number of Comments')

In [ ]:
# Not really

def find_